In [21]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
from selenium.common.exceptions import StaleElementReferenceException

In [22]:
# CARGAR EXCEL CON LA INFO Y CONVERIIR FECHA A FORMATO DE PYTHON FECHA

ruta = "C:/Users/9 ----- SIG/Downloads/RG_041.xlsx"
df = pd.read_excel(ruta)
# Convertir la columna 'FECHA_GRADO' a datetime, indicando el formato original si es necesario
df["FECHA_GRADO"] = pd.to_datetime(df["FECHA_GRADO"], dayfirst=True, errors="coerce")

# Luego convertirla a string con formato dd/mm/yyyy
df["FECHA_GRADO_STR"] = df["FECHA_GRADO"].dt.strftime("%d/%m/%Y")
df

,ID_REGLA,DESCRIPCION_REGLA,PERIODO_REPORTE_POBLACIONAL,CODIGO_PROGRAMA,NOMBRE_PROGRAMA,PARTICIPANTE,TIPO_DOCUMENTO,NUM_DOCUMENTO,FOLIO,ACTA,AÑO,SEMESTRE,FECHA_GRADO,FECHA_GRADO_STR
0,RG_041,Graduado con acta de grado vacía,201401S,55191,ESPECIALIZACION EN AUDITORIA Y CONTROL FISCAL,SNIES-6003569,CC,39619660,52-1,18,2014,1,2014-03-27,27/03/2014
1,RG_041,Graduado con acta de grado vacía,201401S,54802,ESPECIALIZACION EN AUDITORIA Y CONTROL FISCAL,SNIES-3987285,CC,51826024,52-4,21,2014,1,2014-03-27,27/03/2014
2,RG_041,Graduado con acta de grado vacía,201102S,16242,PSICOLOGIA,SNIES-5273406,CC,52266135,262-1,37332,2011,2,2011-09-02,02/09/2011
3,RG_041,Graduado con acta de grado vacía,200401S,14384,ADMINISTRACION DE EMPRESAS,SNIES-9313021,CC,52356381,2-10,14135,2004,1,2001-02-08,08/02/2001
4,RG_041,Graduado con acta de grado vacía,201401S,52683,INGENIERIA DE SISTEMAS,SNIES-6646545,CC,53056520,31-2,41908,2014,1,2014-03-27,27/03/2014
5,RG_041,Graduado con acta de grado vacía,200602S,1943,INGENIERIA DE SISTEMAS,SNIES-4707331,CC,66767010,546-2,31721,2006,2,2007-12-14,14/12/2007
6,RG_041,Graduado con acta de grado vacía,201402S,53438,JUSTICIA Y DERECHO,SNIES-7362646,CC,74182226,303-12,42993,2014,2,2014-10-24,24/10/2014
7,RG_041,Graduado con acta de grado vacía,201402S,53438,JUSTICIA Y DERECHO,SNIES-4441599,CC,74373847,303-5,42986,2014,2,2014-10-24,24/10/2014
8,RG_041,Graduado con acta de grado vacía,201402S,54802,ESPECIALIZACION EN AUDITORIA Y CONTROL FISCAL,SNIES-13184,CC,79050646,244-2,26,2014,2,2014-09-25,25/09/2014
9,RG_041,Graduado con acta de grado vacía,200402S,8936,DERECHO,SNIES-9250627,CC,79125615,352-2,363,2004,2,2007-09-06,06/09/2007


In [35]:
# Crear el navegador
driver = webdriver.Chrome()
# Abrir la página de inicio de sesión
driver.get("https://hecaa.mineducacion.gov.co/hecaa-snies/content/admin/login/login.jsf")
time.sleep(1)
# Ingresar usuario y contraseña
usuario = driver.find_element(By.CSS_SELECTOR, "input[id='loginForm:login-user']")
usuario.send_keys("1826-admin")
contrasena = driver.find_element(By.CSS_SELECTOR, "input[id='loginForm:login-password']")
contrasena.send_keys("1826Uan9123")
# Clic en el botón de ingresar
boton_ingresar = driver.find_element(By.ID, "loginForm:submitLogin")
boton_ingresar.click()
# Ir a la sección administrador - verificar graduados
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "menu_33"))).click()
time.sleep(0.3)
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "menu_37"))).click()
time.sleep(0.3)

# Inicializar columna de estado si no existe
if "ESTADO" not in df.columns:
    df["ESTADO"] = "Pendiente"

# Procesar cada fila individualmente
for index, fila in df.iterrows():
    estado = "Error"  # ✅ Resetear estado para cada fila
    
    try:
        print(f"\n🔁 Procesando fila {index + 1}/{len(df)} - Documento: {fila['NUM_DOCUMENTO']}")
    
        # Esperar a que la tabla se cargue
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//table[contains(@class, 'table-hover')]"))
        )
        
        # Obtener todas las filas de la tabla
        filas = driver.find_elements(By.XPATH, "//table[contains(@class, 'table-hover')]//tr")
        
        # Limpiar valores desde el Excel
        anio_excel = str(int(fila["AÑO"])).strip()
        semestre_excel = str(int(fila["SEMESTRE"])).strip()
        fecha_grado = fila["FECHA_GRADO_STR"]
        acta_excel = str(int(fila["ACTA"])).strip()
        folio_excel = str(fila["FOLIO"]).strip()
        
        fila_encontrada = False
        for fila_html in filas:
            celdas = fila_html.find_elements(By.TAG_NAME, "td")
            if len(celdas) >= 3:
                if celdas[1].text.strip() == anio_excel and celdas[2].text.strip() == semestre_excel:
                    print(f"✅ Año {anio_excel} y semestre {semestre_excel} encontrados. Haciendo clic en Verificar...")
        
                    try:
                        boton_verificar = fila_html.find_element(By.XPATH, ".//span[text()='Verificar']/ancestor::button")
                        boton_verificar.click()
                        print("🟢 Clic realizado con éxito en el primer Verificar.")
                        fila_encontrada = True
                    except:
                        print("❌ No se pudo hacer clic en 'Verificar' dentro de la fila.")
                    break
        
        if not fila_encontrada:
            print(f"❌ No se encontró la fila para Año {anio_excel} y Semestre {semestre_excel}")
            raise Exception(f"No se encontró año {anio_excel} y semestre {semestre_excel}")
        
        # --- 2. Buscar pro_consecutivo y hacer clic en Verificar ---
        pro_consecutivo_excel = str(int(fila["CODIGO_PROGRAMA"])).strip()
        
        # Esperar que aparezca al menos una fila con datos en la segunda tabla
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//table[contains(@class, 'table-hover')]//tr[td]"))
        )
        
        encontrado = False
        
        for intento in range(3):
            try:
                # ⚠️ Mover la obtención de filas *dentro* del bucle
                filas_nuevas = driver.find_elements(By.XPATH, "//table[contains(@class, 'table-hover')]//tr")
        
                for fila_html in filas_nuevas:
                    celdas = fila_html.find_elements(By.TAG_NAME, "td")
                    if len(celdas) >= 2:
                        if celdas[1].text.strip() == pro_consecutivo_excel:
                            print(f"✅ PRO_CONSECUTIVO {pro_consecutivo_excel} encontrado. Clic en Verificar...")
        
                            boton_verificar = fila_html.find_element(By.XPATH, ".//span[text()='Verificar']/ancestor::button")
                            boton_verificar.click()
                            print("🟢 Clic realizado con éxito en el segundo Verificar.")
                            encontrado = True
                            break
                break  # Si no hay excepción, sal del bucle de intentos
        
            except StaleElementReferenceException:
                print(f"⚠️ Intento {intento + 1}: Elemento desactualizado. Reintentando...")
                time.sleep(0.7)
        
        if not encontrado:
            print(f"❌ No se encontró PRO_CONSECUTIVO = {pro_consecutivo_excel} en la segunda tabla")
            
            raise Exception(f"No se encontró PRO_CONSECUTIVO {pro_consecutivo_excel}")
        
        # Esperar que aparezca la tabla con las personas
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//tbody[@id='form:checkboxDT_data']//tr"))
        )
        
        # Leer número de documento desde Excel
        documento_excel = str(int(fila["NUM_DOCUMENTO"])).strip()
        encontrado = False
        
        # Buscar en cada fila
        filas_documentos = driver.find_elements(By.XPATH, "//tbody[@id='form:checkboxDT_data']//tr")
        
        for fila_html in filas_documentos:
            celdas = fila_html.find_elements(By.TAG_NAME, "td")
            if len(celdas) >= 4:  # Nos aseguramos de que exista celda[3]
                documento_actual = celdas[3].text.strip()
        
                if documento_actual == documento_excel:
                    print(f"✅ Documento {documento_excel} encontrado en la tabla. Clic en 'Actualizar información'...")
        
                    try:
                        # Buscar y hacer clic en el botón de 'Actualizar información' dentro de la misma fila
                        boton_actualizar = fila_html.find_element(By.XPATH, ".//span[text()='Actualizar información']/ancestor::button")
                        boton_actualizar.click()
                        print("🟢 Clic en 'Actualizar información' realizado con éxito.")
                        encontrado = True
                    except:
                        print("❌ No se pudo hacer clic en 'Actualizar información'.")
                    break
        
        if not encontrado:
            print(f"❌ No se encontró PRO_CONSECUTIVO = {pro_consecutivo_excel} en la segunda tabla")
        
            raise Exception(f"No se encontró PRO_CONSECUTIVO {pro_consecutivo_excel}")
        # Actualizar fecha de grado
        try:
            # Intentar múltiples veces en caso de elementos stale
            for intento_fecha in range(3):
                try:
                    # Re-encontrar el elemento en cada intento
                    input_fecha = WebDriverWait(driver, 10).until(
                        EC.element_to_be_clickable((By.ID, "formModal:fechaGrado_input"))
                    )
                
                    # Limpiar y escribir la fecha
                    input_fecha.clear()
                    time.sleep(0.2)  # Pequeña pausa después de limpiar
                    input_fecha.send_keys(fecha_grado)
                    
                    # ✅ IMPORTANTE: Hacer clic fuera del input para cerrar el calendario
                    driver.execute_script("arguments[0].blur();", input_fecha)
                    # O hacer clic en otro elemento para quitar el foco
                    try:
                        driver.find_element(By.TAG_NAME, "body").click()
                    except:
                        pass
                    
                    # Esperar un momento para que se cierre el calendario
                    time.sleep(1)
                    
                    print(f"📅 Fecha de grado '{fecha_grado}' ingresada con éxito.")
                    break  # Si llegamos aquí, salir del bucle de intentos
                    
                except StaleElementReferenceException:
                    print(f"⚠️ Intento {intento_fecha + 1}/3: Elemento de fecha desactualizado, reintentando...")
                    time.sleep(1)
                    if intento_fecha == 2:  # Último intento
                        raise Exception("No se pudo ingresar la fecha después de 3 intentos")
                except Exception as e:
                    if intento_fecha == 2:  # Último intento
                        raise e
                    print(f"⚠️ Error en intento {intento_fecha + 1}/3: {e}")
                    time.sleep(1)
        
        except Exception as e:
            print(f"❌ Error al ingresar la fecha de grado: {e}")
            raise e

        # Acta 
        input_acta = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, "formModal:acta"))
        )
    
        # Limpiar y escribir el acta
        input_acta.clear()
        time.sleep(0.2)  # Pequeña pausa después de limpiar
        input_acta.send_keys(acta_excel)

        # Folio 
        input_folio = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, "formModal:folio"))
        )
    
        # Limpiar y escribir el folio
        input_folio.clear()
        time.sleep(0.2)  # Pequeña pausa después de limpiar
        input_folio.send_keys(folio_excel)

        # Hacer clic en actualizar
        try:
            # Intentar múltiples veces en caso de elementos stale
            for intento_boton in range(3):
                try:
                    # Esperar que desaparezca el calendario si está visible
                    try:
                        WebDriverWait(driver, 3).until(
                            EC.invisibility_of_element_located((By.CLASS_NAME, "ui-datepicker-calendar"))
                        )
                    except:
                        pass  # Si no hay calendario, continuar
                    
                    # Re-encontrar el botón en cada intento
                    boton_actualizar_final = WebDriverWait(driver, 10).until(
                        EC.element_to_be_clickable((By.ID, "formModal:j_idt168"))
                    )
                    
                    # Scroll para asegurar que el botón esté visible
                    driver.execute_script("arguments[0].scrollIntoView(true);", boton_actualizar_final)
                    time.sleep(0.5)
                    
                    # Intentar clic normal primero
                    try:
                        boton_actualizar_final.click()
                        print("💾 Clic en el botón 'Actualizar' realizado con éxito.")
                    except:
                        # Si falla, usar JavaScript
                        driver.execute_script("arguments[0].click();", boton_actualizar_final)
                        print("💾 Clic en el botón 'Actualizar' realizado con JavaScript.")
                    
                    break  # Si llegamos aquí, salir del bucle de intentos
                    
                except StaleElementReferenceException:
                    print(f"⚠️ Intento {intento_boton + 1}/3: Botón actualizar desactualizado, reintentando...")
                    time.sleep(1)
                    if intento_boton == 2:  # Último intento
                        raise Exception("No se pudo hacer clic en Actualizar después de 3 intentos")
                except Exception as e:
                    if intento_boton == 2:  # Último intento
                        raise e
                    print(f"⚠️ Error en intento {intento_boton + 1}/3: {e}")
                    time.sleep(1)
        
        except Exception as e:
            print(f"❌ Error al hacer clic en el botón 'Actualizar': {e}")
            raise e
        
        # Paso 1: Volver desde tabla de personas
        try:
            boton_volver = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((
                    By.XPATH,
                    "//a[contains(@onclick, 'mojarra.jsfcljs') and contains(@class, 'btn-default')]"
                ))
            )
            driver.execute_script("arguments[0].scrollIntoView(true);", boton_volver)
            time.sleep(0.3)
            driver.execute_script("arguments[0].click();", boton_volver)
            print("🔙 Clic en el botón 'Volver' realizado con JavaScript.")
        except Exception as e:
            print(f"❌ No se pudo hacer clic en el botón 'Volver': {e}")
            
        # Paso 2: Volver desde tabla de carreras
        try:
            boton_volver_carreras = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//button[.//span[text()='Volver']]"))
            )
            boton_volver_carreras.click()
            print("🔙 Clic en 'Volver' desde carreras.")
        except Exception:
            try:
                driver.execute_script("arguments[0].click();", boton_volver_carreras)
                print("🔙 Clic en 'Volver' desde carreras con JavaScript.")
            except Exception as e:
                print(f"❌ No se pudo hacer clic en 'Volver' desde carreras: {e}")
        # ✅ ¡Si Todo fue exitoso!
        estado = "OK"

    # SI HAY ERROR O NO ENCUENTRA UNA CEDULA O PRO CONSECUTIVO
    
    except Exception as e:
        print(f"⚠️ Error procesando la fila {index + 1}: {e}")
        estado = "Error"
        
        # ✅ Intentar navegar de vuelta sin ir al login
        try:
            print("🔄 Intentando volver a la página de verificación...")
            
            # Primero intentar hacer clic en los botones de Volver si existen
            try:
                # Buscar botón Volver de personas
                boton_volver = driver.find_element(By.XPATH, "//a[contains(@class, 'btn-default') and contains(@onclick, 'mojarra.jsfcljs')]")
                driver.execute_script("arguments[0].click();", boton_volver)
                time.sleep(0.5)
                print("🔙 Regresado desde tabla de personas")
            except:
                pass
                
            try:
                # Buscar botón Volver de carreras
                boton_volver2 = driver.find_element(By.XPATH, "//span[text()='Volver']/ancestor::button")
                driver.execute_script("arguments[0].click();", boton_volver2)
                time.sleep(0.5)
                print("🔙 Regresado desde tabla de carreras")
            except:
                pass
            
            # Si los botones Volver no funcionan, navegar directamente
            current_url = driver.current_url
            if "login" in current_url:
                print("⚠️ El sistema nos llevó al login, saltando fila...")
            else:
                # Verificar si estamos en la página correcta
                try:
                    WebDriverWait(driver, 5).until(
                        EC.presence_of_element_located((By.XPATH, "//table[contains(@class, 'table-hover')]"))
                    )
                    print("✅ Ya estamos en la página correcta")
                except:
                    # Si no estamos en la página correcta, navegar al menú
                    try:
                        WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.ID, "menu_33"))).click()
                        time.sleep(0.3)
                        WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.ID, "menu_37"))).click()
                        time.sleep(0.3)
                        print("✅ Navegación exitosa de vuelta al menú")
                    except:
                        print("❌ No se pudo navegar de vuelta, continuando con la siguiente fila")
            
        except Exception as nav_error:
            print(f"❌ Error en navegación de recuperación: {nav_error}")
            print("⚠️ Continuando con la siguiente fila...")
    
    finally:
        # ✅ Guardar el estado para esta fila específica
        df.at[index, "ESTADO"] = estado

# ✅ Guardar el resultado final
# Cerrar el navegador
driver.quit()
print("Navegador Cerrado")
df[["NUM_DOCUMENTO", "CODIGO_PROGRAMA", "ESTADO"]].to_excel("resultado_procesamiento.xlsx", index=False)
print("📄 Archivo 'resultado_procesamiento.xlsx' guardado con éxito.")
print(f"📊 Resumen: {len(df[df['ESTADO'] == 'OK'])} exitosos, {len(df[df['ESTADO'] == 'Error'])} con errores")


🔁 Procesando fila 1/12 - Documento: 39619660
✅ Año 2014 y semestre 1 encontrados. Haciendo clic en Verificar...
🟢 Clic realizado con éxito en el primer Verificar.
⚠️ Intento 1: Elemento desactualizado. Reintentando...
✅ PRO_CONSECUTIVO 55191 encontrado. Clic en Verificar...
🟢 Clic realizado con éxito en el segundo Verificar.
✅ Documento 39619660 encontrado en la tabla. Clic en 'Actualizar información'...
🟢 Clic en 'Actualizar información' realizado con éxito.
📅 Fecha de grado '27/03/2014' ingresada con éxito.
💾 Clic en el botón 'Actualizar' realizado con éxito.
🔙 Clic en el botón 'Volver' realizado con JavaScript.
🔙 Clic en 'Volver' desde carreras.

🔁 Procesando fila 2/12 - Documento: 51826024
✅ Año 2014 y semestre 1 encontrados. Haciendo clic en Verificar...
🟢 Clic realizado con éxito en el primer Verificar.
⚠️ Intento 1: Elemento desactualizado. Reintentando...
❌ No se encontró PRO_CONSECUTIVO = 54802 en la segunda tabla
⚠️ Error procesando la fila 2: No se encontró PRO_CONSECUTIVO 